In [24]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.model_selection import GridSearchCV

In [10]:
df = pd.read_csv("Speeddating_Processed.csv")
df = df.drop(["Unnamed: 0", "has_null", "wave", "interests_correlate", "guess_prob_liked", "decision", "decision_o", "met"], axis=1)
df = df.drop([col for col in df if col.startswith("d_")], axis=1)
df = df.drop([col for col in df if "_o" in col], axis=1)

for ind, row in df.iterrows():
    if row["gender"] == "female":
        df.loc[ind, "gender"] = 1
    elif row["gender"] == "male":
        df.loc[ind, "gender"] = 0

# df = df.join(pd.get_dummies(df.pop("race"), dtype=int))
df = df.drop("race", axis=1)

# Field Column One Hot Encoded or Dropped
# df = df.join(pd.get_dummies(df.pop("field"), dtype=int))
df = df.drop("field", axis=1)

df.head()

,gender,age,samerace,importance_same_race,importance_same_religion,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,...,tv,theater,movies,concerts,music,shopping,yoga,expected_happy_with_sd_people,like,match
0,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,7.0,0
1,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,7.0,0
2,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,7.0,1
3,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,6.0,1
4,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,6.0,0


In [11]:
#Withhold Binary columns from regularization
tempDF = df.drop(["match", "gender", "samerace"], axis=1)

scaler = StandardScaler()
scaled_arr = scaler.fit_transform(tempDF)
scaled_df = pd.DataFrame(scaled_arr, index=tempDF.index, columns=tempDF.columns).join(df.loc[:,["match", "gender", "samerace"]])

scaled_df

,age,importance_same_race,importance_same_religion,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,attractive,...,movies,concerts,music,shopping,yoga,expected_happy_with_sd_people,like,match,gender,samerace
0,-1.510659,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,0.493945,-0.787565,...,1.235556,1.477397,0.651220,0.880210,-1.244794,-1.455416,0.465329,0,1,0
1,-1.510659,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,0.493945,-0.787565,...,1.235556,1.477397,0.651220,0.880210,-1.244794,-1.455416,0.465329,0,1,0
2,-1.510659,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,0.493945,-0.787565,...,1.235556,1.477397,0.651220,0.880210,-1.244794,-1.455416,0.465329,1,1,0
3,-1.510659,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,0.493945,-0.787565,...,1.235556,1.477397,0.651220,0.880210,-1.244794,-1.455416,-0.086090,1,1,0
4,-1.510659,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,0.493945,-0.787565,...,1.235556,1.477397,0.651220,0.880210,-1.244794,-1.455416,-0.086090,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7074,-0.371125,-0.982478,-0.943086,4.046668,-2.518729,-0.797401,-0.411495,-1.767049,-1.873438,0.652676,...,0.627569,1.477397,1.222117,0.496882,-0.506253,2.591509,-1.188927,0,0,1
7075,-0.371125,-0.982478,-0.943086,4.046668,-2.518729,-0.797401,-0.411495,-1.767049,-1.873438,0.652676,...,0.627569,1.477397,1.222117,0.496882,-0.506253,2.591509,-2.291764,0,0,0
7076,-0.371125,-0.982478,-0.943086,4.046668,-2.518729,-0.797401,-0.411495,-1.767049,-1.873438,0.652676,...,0.627569,1.477397,1.222117,0.496882,-0.506253,2.591509,-1.188927,0,0,0
7077,-0.371125,-0.982478,-0.943086,4.046668,-2.518729,-0.797401,-0.411495,-1.767049,-1.873438,0.652676,...,0.627569,1.477397,1.222117,0.496882,-0.506253,2.591509,-0.637508,0,0,0


In [12]:
X = scaled_df.drop("match", axis=1)
y = scaled_df["match"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [13]:
model = svm.SVC()

parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[1, 10]}
CVmodel = GridSearchCV(model, parameters)

CVmodel.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10],
                         'kernel': ('linear', 'poly', 'rbf', 'sigmoid')})

In [38]:
CVmodel.best_params_

{'C': 10, 'kernel': 'rbf'}

In [14]:
preds = CVmodel.predict(X_test)
acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds)
rec = recall_score(y_test, preds)

acc, f1, rec

(0.8350282485875706, 0.4065040650406504, 0.31746031746031744)